### _Nothing in Biology Makes Sense Except in the Light of Evolution_ 
###### Theodosius  Dobzhansky

# TP-1: alineamiento global con Needleman Wunsch

Escriba una implementación en Python del algoritmo de alineamiento global de Needleman-Wunsch, 
dentro de las 4 celdas indicadas más abajo.

El alineamiento entre las 2 secuencias de `ab.fasta` debe tener un score de 158 y
el alineamiento entre las 2 secuencias de `ae.fasta` debe tener un score de 312.

--------

In [1]:
import numpy as np
from Bio import pairwise2, SeqIO
from enum import Enum

## Needleman-Wunsch con BioPython

In [2]:
aln = pairwise2.align.globalxx("TATA", "TCT")
aln[0].score

2.0

In [3]:
print(aln[0].seqA)
print(aln[0].seqB)

TA-TA
T-CT-


In [4]:
f_aln = SeqIO.parse("ab.fasta", 'fasta')
fas_A = next(f_aln)
fas_B = next(f_aln)

seq_A = str(fas_A.seq)
seq_B = str(fas_B.seq)

aln = pairwise2.align.globalxx(seq_A, seq_B)
aln[0].score

158.0

In [5]:
print(aln[0].seqA[300:])
print(aln[0].seqB[300:])

CGTCA-AAGATCTC-CAA-CCCGGGGACAGAGT-ACTGGCCGCGGATT-ACGACGGAA--ACCCGGTTTATACCGACTTCATCATGTTCAA-
C-TC-CAA-AT-T-ACAATCCC---GAC--A-TTA-T-----C---TTTA--A-GG-ATGA---GG-----A--GA----A-CA----C--G


In [6]:
f_aln = SeqIO.parse("ae.fasta", 'fasta')
fas_A = next(f_aln)
fas_E = next(f_aln)

seq_A = str(fas_A.seq)
seq_E = str(fas_E.seq)

aln = pairwise2.align.globalxx(seq_A, seq_E)
aln[0].score

312.0

In [7]:
print(aln[0].seqA[300:])
print(aln[0].seqB[300:])

GGCCCG-TCAAAGATC-TCCAACCCGGG-GACAGAG-TAC-TGGCC-GCG-GATT-ACGA-CGGAAACCCGG-TTTAT-ACCGACTTCATCATGTTCAA
GG--CGGTCAAAGA-CCTCCAACCC-GGCGACAG-GGT-CCTGG-CAGC-AGA-TGA-GAACGGAAACCC-GATTTA-CACCGAC-TC--C---TT---


------

## Needleman-Wunsch propio

#### Cosas q pueden ser útiles

### 1
`op` es un type que representa las 3 operaciones posibles en un alineamiento.

In [8]:
class op(Enum):
    GAP_A = 1
    GAP_B = 2
    MA_MM = 3

In [9]:
class score_match_miss_match():
    
    def __init__(self):
        self.matrix = np.matrix([[1 , 0 , 0 , 0], 
                                 [0 , 1 , 0 , 0], 
                                 [0 , 0 , 1 , 0], 
                                 [0 , 0 , 0 , 1]])
    
    def parse_nucleotide(self, nucleotide):
        if nucleotide == 'A':
            return 0
        if nucleotide == 'T':
            return 1
        if nucleotide == 'G':
            return 2
        if nucleotide == 'C':
            return 3
    
    def get_score(self, nucleotide_A, nucleotide_B):
        code_n_a = self.parse_nucleotide(nucleotide_A)
        code_n_b = self.parse_nucleotide(nucleotide_B)
        return self.matrix[code_n_a,code_n_b]
    





In [10]:
matrix = score_match_miss_match()
matrix.get_score('A', 'C')

0

### 2
`score_mtx` contiene los scores de match y mismatch entre las distintas bases.
Para reproducir el comportamiento de la función`pairwise2.align.globalxx()` de biopython
asignaremos, por ahora, un score de `1` a cualquier match y `0` a cualquier otro match.

El gap score irá por fuera de `score_mtx` y será, circunstancialmente, de `0`.

`score_mtx` puede ser una lista de listas, `numpy.darray`, diccionario, o lo que crea conveniente.

In [11]:
score_mtx = np.repeat(-1, (4) * (4)).reshape(4, 4)
for i in range(0,4):
    score_mtx[i][i]=1
score_mtx

array([[ 1, -1, -1, -1],
       [-1,  1, -1, -1],
       [-1, -1,  1, -1],
       [-1, -1, -1,  1]])

### 3
`traceback()` toma la tabla donde `nw()` memoizó los subproblemas (`tabla_sol`),
las 2 secuencias originales (`seqA` y `seqB`) y entrega las 2 secuencias modificadas (`aln_a` y `aln_b`),
que puestas una arriba de la otra, mostrarían un alineamiento correcto.

In [12]:
def traceback(matrix_score, seqA, seqB):    
    n, m = camino_mas_corto(matrix_score, 0,0)
    m.reverse()
    return align(seqA, seqB,m,0, 0, 0)
    
def align(seqA, seqB, operation_list, idx_operations,x,y):
    w = len(seqA)
    z = len(seqB)
    if (x == w and z == y):
        return '', ''
    if x == w:
        return '', seqB[y:]
    if y == z:
        return seqA[x:], ''
    i = operation_list[idx_operations]
    
    if i == 'Match':
        aln_a_res, aln_b_res= align(seqA,seqB,operation_list,idx_operations+1,x+1,y+1)
        return seqA[x] + aln_a_res, seqB[y] + aln_b_res
    if i == 'gap_A':
        aln_a_res, aln_b_res= align(seqA,seqB,operation_list,idx_operations+1,x,y+1)
        return '-' + aln_a_res, seqB[y]  + aln_b_res
    if i == 'gap_B':
        aln_a_res, aln_b_res= align(seqA,seqB,operation_list,idx_operations+1,x+1,y)
        return seqA[x]  + aln_a_res, '-' + aln_b_res
            
# No performante.... revisarlo
def camino_mas_corto(matrix_score, x, y):
    # Caso base
    m, n = matrix_score.shape
    if(x == m and y == n):
        return 0, []
    if(x == m-1):
        return n-y-1, [ 'gap_A' for x in range(n-1-y)]
    if(y == n-1):
        return m-x-1,[ 'gap_B' for x in range(m-1-x)]
    #Camino gaps
    if matrix_score[x][y] == matrix_score[x+1][y+1]:
        mi, cam   = camino_mas_corto(matrix_score,x+1,y)
        mi2, cam2 = camino_mas_corto(matrix_score,x,y+1)
        if mi < mi2:
            return mi+1, cam + ['gap_A']
        return mi2+1, cam2 + ['gap_B']
    #Camino match o missmatch
    mchta, camchta = camino_mas_corto(matrix_score,x+1,y+1)
    return mchta + 1, camchta + ['Match']


In [13]:
[ 'gap' for x in range(4)]

['gap', 'gap', 'gap', 'gap']

### 4
`nw()` implementa el algoritmo de needleman-wunsch. Toma las 2 secuencias en formato de `str` (`seqA_str` y `seqB_str`),
para igualar el comportamiento de `pairwise2.align.globalxx()`, la `score_mtx` y un `gap_score`.
Debe llamar a `traceback()` para luego devolver las 2 secuencias alineadas y el score del alineamiento (último elemento de `tabla_score`)

In [14]:
def nw(seqA_str, seqB_str, score_mtx, gap_score = 0):
    l_seqA = len(seqA_str)
    l_seqB = len(seqB_str)
    m = l_seqA + 1
    n = l_seqB + 1
    matrix_score =  np.repeat(0, m * n).reshape(m, n)


    for i in range(1,m):
        for j in range(1,n):
            if seqA_str[i-1] == seqB_str[j-1]:
                matrix_score[i][j] = matrix_score[i-1][j-1] + score_mtx.get_score(seqA_str[i-1], seqB_str[j-1])
            else:
                matrix_score[i][j] = max(matrix_score[i][j-1] + gap_score,          # Gap_B
                                       matrix_score[i-1][j] + gap_score ,            # Gap_A
                                       matrix_score[i-1][j-1] + score_mtx.get_score(seqA_str[i-1], seqB_str[j-1]))      # MissMatch
    print(matrix_score)
    als = traceback(matrix_score, seqA_str, seqB_str)
    return als[0],als[1], matrix_score[m-1][n-1]


In [15]:
nw('TATA', "TCT", score_match_miss_match(),0)

[[0 0 0 0]
 [0 1 1 1]
 [0 1 1 1]
 [0 1 1 2]
 [0 1 1 2]]


('TA-TA', 'T-CT', 2)

-------

## Pruebe su implementación

In [16]:
# Toma de input
f_aln = SeqIO.parse("ab.fasta", 'fasta')
fas_A = next(f_aln)
fas_B = next(f_aln)

seq_A = str(fas_A.seq)
seq_B = str(fas_B.seq)

In [17]:
score_mtx_1 = score_match_miss_match()

In [ ]:
# Confirme el score
aln_a, aln_b, score = nw(seq_A, seq_B, score_mtx_1)
score

[[  0   0   0 ...   0   0   0]
 [  0   0   0 ...   1   1   1]
 [  0   1   1 ...   2   2   2]
 ...
 [  0   1   2 ... 157 158 158]
 [  0   1   2 ... 157 158 158]
 [  0   1   2 ... 157 158 158]]


In [ ]:
# Visualice el alineamiento. Recuerde que no es necesario que sea idéntico al de Biopython
str_aln_a = str().join(aln_a)
str_aln_b = str().join(aln_b)

print(str_aln_a[300:])
print(str_aln_b[300:])

In [ ]:
# Toma de input
f_aln = SeqIO.parse("ae.fasta", 'fasta')
fas_A = next(f_aln)
fas_E = next(f_aln)

seq_A = str(fas_A.seq)
seq_E = str(fas_E.seq)

In [ ]:
# Confirme el score
aln_a, aln_e, score = nw(seq_A, seq_E, score_mtx_1)
score

In [ ]:
# Visualice el alineamiento. Recuerde que no es necesario que sea idéntico al de Biopython
str_aln_a = str().join(aln_a)
str_aln_e = str().join(aln_e)

print(str_aln_a[300:])
print(str_aln_e[300:])